In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/2022-2023 Awards Project clean/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 2 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

31 October 2022 ; 11:05:26


In [3]:
features = ['DWS','DBPM','STL','BLK','DRB','PA/G','BLK%','DRtg/100','MP']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/dpoy/dpoy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,6,0,23.2,3.0,8.5,0.353,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1
1,OG Anunoby,SF,25,TOR,6,6,36.2,4.7,10.8,0.431,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,0
2,Dalano Banton,PG,23,TOR,6,0,8.2,1.0,2.5,0.400,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1
3,Scottie Barnes,PF,21,TOR,5,5,28.6,5.8,11.0,0.527,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,0
4,Khem Birch,C,30,TOR,2,0,6.5,0.5,1.0,0.500,...,-1.33,110.95,112.39,-1.44,0.2,111.25,110.99,0.26,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
import matplotlib.pyplot as plt
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'DBPM', 'STL', 'BLK', 'DRtg/100', 'DWS', 'Seed', 'Share']]
res.head(5)

,Player,Tm,DBPM,STL,BLK,DRtg/100,DWS,Seed,Share
0,Giannis Antetokounmpo,MIL,5.1,0.6,1.6,98.0,8.200000,1,0.549729
1,Brook Lopez,MIL,3.0,0.8,3.6,99.0,6.560000,1,0.335189
2,Jrue Holiday,MIL,2.0,2.0,1.2,101.0,6.560000,1,0.217544
3,Caris LeVert,CLE,0.4,1.2,0.2,107.0,5.466667,2,0.179459
4,Donovan Mitchell,CLE,1.4,1.7,0.5,105.0,5.466667,2,0.178732


In [7]:
res.to_csv(path + f'Results/dpoy/results_week_{week}.csv', index = None)